In [1]:
import pandas as pd
import matplotlib as mpl
from datetime import *

In [2]:
#Reading CSV file
df = pd.read_csv("C:\\Users\\karth\\glassdoors_dataset.csv")

In [3]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [4]:
#Removing 1st Column named Unnamed
df = df.iloc[: , 1:]

In [5]:
#Create Hourly column where the income is per_hour or not
df['Hourly'] = df['Salary Estimate'].apply(lambda x:1 if "per hour" in x.lower() else 0)
#Create Employer_Provided_Salary column where 1 states yes and 0 states no
df['Employer_Provided_Salary'] = df['Salary Estimate'].apply(lambda x:1 if "employer provided" in x.lower() else 0)

In [6]:
found = df['Salary Estimate'].str.contains('-1')
print(found.count())

956


In [7]:
#Filtering rows in the dataframe where the Salary Estimate rows has -1 string in it
df = df[df['Salary Estimate']!='-1']

In [8]:
#Removing the words after the salary range
income = df['Salary Estimate'].apply(lambda y: y.split('(')[0])
#Replacing $ and K with no space
income = income.apply(lambda y: y.replace('$','').replace('K','')) 
#Replace employer provided salary: and per hour with no space
income_hr = income.apply(lambda y: y.lower().replace('employer provided salary:','').replace('per hour',''))

In [9]:
#Taking and Multiplying doubling the maximum income if role has hourly wages
df['maximum_income'] = income_hr.apply(lambda y: int(y.split('-')[1]))
df['maximum_income'] = df.apply(lambda x: x.maximum_income*2 if x.Hourly == 1 else x.maximum_income, axis = 1)

In [10]:
#Taking and Multiplying doubling the minimum income if role has hourly wages
df['minimum_income'] = income_hr.apply(lambda y: int(y.split('-')[0]))
df['minimum_income'] = df.apply(lambda x: x.minimum_income*2 if x.Hourly == 1 else x.minimum_income, axis = 1)

In [11]:
#Calculating average income if role has hourly wages
df['average_income'] = (df['minimum_income']+df['maximum_income'])/2
df['average_income'] = df.apply(lambda x: (x.minimum_income+x.maximum_income)/2 if x.Hourly == 1 else x.average_income, axis = 1)

In [12]:
#Calculating the Companys' age
df['Company_Age'] = df['Founded'].apply(lambda x: int(2020-x))
df['Company_Age'] = df['Company_Age'].replace(to_replace = 2021,value= -1)

In [13]:
df['Location'].unique()

array(['Albuquerque, NM', 'Linthicum, MD', 'Clearwater, FL',
       'Richland, WA', 'New York, NY', 'Dallas, TX', 'Baltimore, MD',
       'San Jose, CA', 'Rochester, NY', 'Chantilly, VA', 'Plano, TX',
       'Seattle, WA', 'Cambridge, MA', 'Newark, NJ', 'Mountain View, CA',
       'San Francisco, CA', 'Denver, CO', 'Chicago, IL', 'Louisville, KY',
       'Herndon, VA', 'Hillsboro, OR', 'Worcester, MA', 'Groton, CT',
       'Detroit, MI', 'Sunnyvale, CA', 'Ipswich, MA', 'Redlands, CA',
       'Woburn, MA', 'Fremont, CA', 'Long Beach, NY', 'Marlborough, MA',
       'Allendale, NJ', 'Washington, DC', 'Bellevue, WA', 'Longmont, CO',
       'Beavercreek, OH', 'Peoria, IL', 'Fort Lauderdale, FL',
       'Boston, MA', 'Huntsville, AL', 'Armonk, NY', 'San Diego, CA',
       'Saint Louis, MO', 'Cincinnati, OH', 'Palo Alto, CA',
       'Coraopolis, PA', 'Framingham, MA', 'Atlanta, GA',
       'Philadelphia, PA', 'Vancouver, WA', 'Indianapolis, IN',
       'Lake Forest, IL', 'Maryland Heights, MO

In [14]:
#Splitting State and City
df['State'] = df['Location'].apply(lambda y: y.split(",")[1])
df['State'] = df['State'].apply(lambda y: y.strip() if y.strip().lower()!= "los angeles" else "CA")
df['City'] = df['Location'].apply(lambda y: y.split(",")[0])

In [15]:
#Location of the Job
df['location_job'] = df.apply(lambda y: 1 if y.Location == y.Headquarters else 0, axis  = 1)

In [16]:
#Companys' name
df['Company_name'] = df.apply(lambda y: y['Company Name'] if y['Rating'] <0 else y['Company Name'][:-3], axis = 1)
df['Company_name'] = df['Company_name'].apply(lambda y: y.split("\n")[0])

In [17]:
#Calculating the number of competitors
df['Competitors_Number'] = df['Competitors'].apply(lambda y: len(y.split(",")) if y!='-1' else 0)

In [18]:
#Clength of the job
df['Job_Length'] = df['Job Description'].apply(lambda x:len(x))

In [19]:
#What are the languages required for the job
df['Python'] = df['Job Description'].apply(lambda x: 1 if "python" in x.lower() else 0)
df['R_Stdio'] = df['Job Description'].apply(lambda x: 1 if "r studio" in x.lower() else 0)
df['R_Stdio'] = df['Job Description'].apply(lambda x: 1 if " R " in x else 0)
df['AWS'] = df['Job Description'].apply(lambda x: 1 if "aws" in x.lower() else 0)
df['Spark'] = df['Job Description'].apply(lambda x: 1 if "spark" in x.lower() else 0)
df['Excel'] = df['Job Description'].apply(lambda x: 1 if "excel" in x.lower() else 0)

In [20]:
#Rephrasing the job title
def job_title_rephrase(job_title):
    
    if "data engineer" in job_title.lower():
        return "Data Engineer"
    
    elif "data scientist" in job_title.lower():
        return "Data Scientist"
    
    elif "manager" in job_title.lower():
        return "Manager"
    
    elif "director" in job_title.lower():
        return "Director"
    
    elif "analyst" in job_title.lower():
        return "Analyst"
  
    elif "machine learning" in job_title.lower():
        return "Machine Learning Engineer";
    
    else:
        return "Other Titles"

In [21]:
#Senior or Junior Role
def seniority(title):
    
    if 'SR' in title.upper() or 'sr.' in title.lower() or 'SENIOR' in title.upper() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'Senior'
   
    elif 'JR' in title.upper() or 'jr.' in title.lower() or 'JUNIOR' in title.upper():
        return 'Junior'
    
    else:
        return 'Not Mentioned'

In [22]:
df['Jobs_Rephrases'] = df['Job Title'].apply(job_title_rephrase)
df['Seniority'] = df['Job Title'].apply(seniority)

In [23]:
df['L_Job_Description'] = df['Job Description'].apply(lambda y:len(y))

In [24]:
df.to_csv('clean_glassdoors_dataset.csv',index = False)

In [25]:
df.shape

(742, 34)

In [26]:
df = df.append([df]*130)

In [27]:
df.shape

(97202, 34)

In [28]:
df.to_csv('mclean_glassdoors_dataset.csv',index = False)